# Dataset Generator

The models we design are only as good as the data that is fed into them. No matter how well a model or network is designed the efficacy of its predictions will be limited by the integrity of its data, "garbage in, garbage out". Since no open-access dataset exists sufficiently large enough to suit our needs we created our own. The code in this notebook will allow you to interactively create a dataset using the same process we used. To generate an identical dataset leave all parameters at their default settings.

If running this notebook from Google Colab you will need to perform a few additional steps first.

### Install Dependencies

In [ ]:
!pip install pydub
!pip install dask[dataframe]
!pip install --upgrade matplotlib

Restart the runtime and continue to the next cell.

### Clone Repository

In order to run the code in this notebook you will need to clone [this git repository](https://github.com/JerameyATyler/cepbimo.git). The following steps will walk you through cloning the repo.

#### Mount Google Drive

In [ ]:
# @title Mounting Google Drive
# @markdown We need to mount a Google Drive in order to download the code we need.
# @markdown > Import `drive` from `google.colab`
from google.colab import drive
# @markdown > Mount Google Drive
drive.mount('/content/gdrive')

#### Make Directory for Repository

In [ ]:
# @markdown > Create a directory for the code.
% mkdir "gdrive/MyDrive/Colab Notebooks/"

#### Change Directories

In [ ]:
# @markdown > Change directories to the new one
% cd "gdrive/MyDrive/Colab Notebooks/"

#### Clone the Git Repository

In [ ]:
# @title Cloning Git Repository
# @markdown Next we need to clone the Git repository
# @markdown > Clone the repository
! git clone https://github.com/JerameyATyler/cepbimo.git

In [ ]:
# @markdown > Change directory to the repository
% cd cepbimo/cepbimo
%mkdir data/
%mkdir data/anechoic/
%mkdir data/hrtf/
%mkdir data/reflections
%cd data

In [ ]:
# @title Anechoic Data
# @markdown Anechoic data recorded by [(Pätynen and Lokki, 2008)](#patynen-lokki-08)
# @markdown > Links to anechoic recordings
file_links = [
    'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/beethoven_mp3.zip',
    'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mozart_mp3.zip',
    'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/bruckner_mp3.zip',
    'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mahler_mp3.zip']
# @markdown > Download files if they don't exist
for l in file_links:
    !wget -nc {l}

In [ ]:
# @title HRTF Data
# @markdown HRTF data recorded by [(Gardner & Martin, 1994)](#gardner-martin-94)
!wget -nc "https://sound.media.mit.edu/resources/KEMAR/full.zip"

### Extracting Data

Extract the archived data to the `data/` directory.
***
**Warning: Overwrites existing files**

In [ ]:
# @title Extracting Data
# @markdown Extract zipped data to `data/` directory.
# @markdown > List of files to unzip
zip_files = [
    ['beethoven_mp3.zip', 'anechoic/beethoven'],
    ['mozart_mp3.zip', 'anechoic/mozart'],
    ['bruckner_mp3.zip', 'anechoic/bruckner'],
    ['mahler_mp3.zip', 'anechoic/mahler'],
    ['full.zip', 'hrtf']]
# > Unzip each archive to `data/anechoic` directory
for z in zip_files:
    !unzip -o {z[0]} -d {z[1]}

In [ ]:
%cd ..

Skip to [Data sources](#data-sources) to continue.

## Data sources
<a name='data-sources'><a/>

### Anechoic Data

In order to generate a dataset of reflections we need to start with anechoic data. A dataset of anechoic orchestral recordings is available thanks to [(Pätynen and Lokki, 2008)](#patynen-lokki-08).
***
> 1. [Mozart's An aria of Donna Elvira](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mozart_mp3.zip)
>   - Flute, clarinet, bassoon, french horns (1-2), violin (I), violin (II), viola, cello, contrabass, soprano (soloist)
> 2. [Beethoven's Symphony no. 7, I movement, bars 1-53](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/beethoven_mp3.zip)
>   - Flutes (1-2), oboes (1-2), clarinets (1-2), bassoon (1-2), french horns (1-2),
>     trumpets (1-3), timpani, violin (I), violin (II), viola, cello, contrabass
> 3. [Bruckner's Symphony no. 8, II movement, bars 1-61](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/bruckner_mp3.zip)
>   - Flutes (1-3), oboes (1-3), clarinets (1-3), bassoon (1-3),
>     french horns (1-8), trumpets (1-3), trombones (1-3), tuba, timpani,
>     violin (I) (two divisi), violin (II) (two divisi), viola (two divisi),
>     cello (two divisi), contrabass (two divisi)
> 4. [Mahler's Symphony no. 1, IV movement, bars 1-85](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mahler_mp3.zip)
>   - Piccolo (1-2) (fl1), flutes (1-2) (fl3), oboes (1-4), clarinets (1-4),
> bassoon (1-3), french horns (1-7), trumpets (1-4), tuba, timpani,
> percussions (1-2), violin I (two divisi), violin II (two divisi),
> viola, cello, contrabass
***
For more information on the anechoic data we use click [this link.](https://research.cs.aalto.fi//acoustics/virtual-acoustics/research/acoustic-measurement-and-analysis/85-anechoic-recordings.html)

### Head-related Transfer Functions

Next we need a dataset of head-related transfer functions (HRTF). A set of HRTFs is available thanks to [(Gardner & Martin, 1994)](#gardner-martin-94) and MIT Media Lab.
***
For more information see [KEMAR HRTF measurements](https://sound.media.mit.edu/resources/KEMAR.html)

## Dataset

In [ ]:
%mv ~/anechoic data
%mv ~/hrtf data

The Python `DataPicker` class will walk you through generating data.

In [1]:
from notebooks.data_picker import DataPicker

dp = DataPicker()

Data generator started
Generating ingredients list


### Configurable parameters

The data generator has a number of parameters that can be set to place restrictions on the generated data. Below are the configurable parameters and their default values.

In [2]:
dp.sample_parameters

,seed,duration,delay_limits,time_limits,reflection_limits,zenith_limits,azimuth_limits,sample_count
0,0xec0ec0,10,"(1, 60)","(1, 8)","(4, 8)","(-40, 90)","(0, 360)",1


> * Seed - A hex value between 0x0000 and 0xffff that is used to initialize the random number generator
> * Duration - The length of each sample in seconds
> * Delay - The amount of time in milliseconds that a reflection or reverberation will lag behind a signal
> * Time - The amount of time in seconds that reverberation will last
> * Reflection count - The number of reflections in a sample
> * Zenith - The elevation angle in degrees from the listener to the direct signal or a reflection
> * Azimuth - The rotation angle in degrees from the listener to the direct signal or a reflection
> * Sample count - The number of samples to generate
> * Verbose - When true all output will be generated

### Sample data

Using the parameters above a sample dataset was generated with a single data point.

In [ ]:
dp.sample_data

Sample data can be found in the `data/sample_data/reflections/` directory. The `reflections/` directory will contain the `recipe.json` and `ingredients.json` files along with a `samples/` directory. The `ingredients.json` file contains the parameters that were used to generate the dataset. The `samples.json` file contains labels for each data point. The `samples/` directory contains `.wav` files generated by the data generator. When `verbose` is set to `True` the data generator will save `.wav` and `.png` files for each step in the generation process. `verbose` increases the disk size of the dataset by approximately 10x. The cell below has a brief overview of the output data.

In [ ]:
dp.sample_viewer()

### Data generator

#### Set parameters

Use the sliders in the cell below to configure the dataset.

> #### Instructions
> 1. Select parameters using the sliders
> 2. Run `Review parameters` code cell and press confirm
> 3. Press the generate button in the `Generate dataset` cell

In [ ]:
dp.picker_viewer()

#### Review parameters

Review the parameters below then press the `Confirm` button. **If changes are made rerun the cell and press confirm**

In [ ]:
dp.review_viewer()

#### Generate dataset

Pressing the `Confirm` button above loads the parameters into the data generator. Press the `Generate` button below to start generating a dataset. When the dataset is complete a `download` link will appear. Additionally a `.zip` archive of the dataset can be found in the `data/` directory.

In [ ]:
dp.generate_viewer()

## References

* Gardner, B., & Martin, K. (1994). Hrtf measurements of a kemar dummy-head microphone. MIT Media Lab. Perceptual Computing-Technical Report, 280, 1-7.
<a name='gardner-martin-94'><a/>

* Pätynen, J., Pulkki, V., & Lokki, T. (2008). Anechoic recording system for symphony orchestra. Acta Acustica united with Acustica, 94(6), 856-865. 
<a name='patynen-lokki-08'><a/>